In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv


## Simple recommendation system

### Simple Recommendation system

In [85]:
#import the required libraries that are required %%latex the recommndation project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from ast import literal_eval
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')
pd.pandas.set_option('display.max_columns',None)

In [3]:
#Read the meta dataset
meta = pd.read_csv('../input/the-movies-dataset/movies_metadata.csv')
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
#Check the shape of the dataset
meta.shape

(45466, 24)

In [5]:
#Check any null values are present in the data
meta.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [6]:
meta['genres'] = meta['genres'].fillna('[]').apply(literal_eval).apply(lambda x : [i['name'] for i in x] if isinstance(x,list) else [])

### Weightes rating method

## Weighted Rating (WR) =  (v/(v+m) * R) + (m/(v+m) * C)

In [7]:
vote_count = meta[meta['vote_count'].notnull()]['vote_count'].astype('int')
vote_avg = meta[meta['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_avg.mean()
C

5.244896612406511

In [8]:
m = vote_count.quantile(0.95)
m

434.0

In [9]:
meta['release_date'][1].split('-')[0]

'1995'

In [10]:
meta['year'] = pd.to_datetime(meta['release_date'],errors='coerce').apply(lambda x : str(x).split('-')[0] if x !=np.nan else np.nan)

In [11]:
qualified = meta[(meta['vote_count'] >= m ) & (meta['vote_count'].notnull()) & (meta['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity','genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')

In [12]:
qualified.head()

,title,year,vote_count,vote_average,popularity,genres
0,Toy Story,1995,5415,7,21.946943,"[Animation, Comedy, Family]"
1,Jumanji,1995,2413,6,17.015539,"[Adventure, Fantasy, Family]"
5,Heat,1995,1886,7,17.924927,"[Action, Crime, Drama, Thriller]"
9,GoldenEye,1995,1194,6,14.686036,"[Adventure, Action, Thriller]"
15,Casino,1995,1343,7,10.137389,"[Drama, Crime]"


Therefore, to qualify to be considered for the chart, a movie has to have at least 434 votes on TMDB. We also see that the average rating for a movie on TMDB is 5.244 on a scale of 10. 2274 Movies qualify to be on our chart.

In [13]:
def weighted_avg(x):
    v = x['vote_count']
    r = x['vote_average']
    return (v/(v+m) * r) + (m/(m+v) * C)

In [14]:
qualified['wr'] = qualified.apply(weighted_avg,axis = 1)

In [15]:
qualified = qualified.sort_values(by = 'wr',ascending=False).head(250)

## Top Movies

In [16]:
qualified.head(10).reset_index(drop=True)

,title,year,vote_count,vote_average,popularity,genres,wr
0,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
1,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
2,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
3,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
5,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
6,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
8,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
9,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


In [17]:
s = meta.apply(lambda x : pd.Series(x['genres']),axis = 1).stack().reset_index(level = 1, drop=True)
s.name = 'genres'
s

0        Animation
0           Comedy
0           Family
1        Adventure
1          Fantasy
           ...    
45461       Family
45462        Drama
45463       Action
45463        Drama
45463     Thriller
Name: genres, Length: 91106, dtype: object

In [18]:
gen_md = meta.drop('genres',axis=1).join(s)

In [19]:
def build_chart_simple(genre, percentile = 0.85):
    df = gen_md[gen_md['genres'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_avg = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_avg.mean()
    m = vote_count.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    def weight_avg(x):
        v = x['vote_count']
        r = x['vote_average']
        return (v/(v+m) * r) + (m/(m+v) * C)
    qualified['wr'] = qualified.apply(weight_avg,axis=1)
    
    qualified = qualified.sort_values(by='wr',ascending=False)
    
    return qualified

## Let's Check our simple recommender system

In [20]:
#Inputs
Genre = str(input('Enter the Genre: ').capitalize())
build_chart_simple(Genre).head(10).reset_index(drop=True)

Enter the Genre:  action


,title,year,vote_count,vote_average,popularity,wr
0,Inception,2010,14075,8,29.108149,7.983618
1,The Dark Knight,2008,12269,8,123.167259,7.981222
2,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.974156
3,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.972084
4,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.969970
5,Star Wars,1977,6778,8,42.149697,7.966192
6,The Empire Strikes Back,1980,5998,8,19.470959,7.961855
7,Scarface,1983,3017,8,11.299673,7.925162
8,Oldboy,2003,2000,8,10.616859,7.888606
9,Seven Samurai,1954,892,8,15.01777,7.761886


# Content Based Recommendation system

In [21]:
link = pd.read_csv('../input/the-movies-dataset/links_small.csv')
link = link[link['tmdbId'].notnull()]['tmdbId'].astype('int')
link.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int64

In [22]:
meta = meta.drop([19730, 29503, 35587])

In [23]:
meta['id'] = meta['id'].astype('int')

In [24]:
smd = meta[meta['id'].isin(link)]

In [25]:
smd.shape

(9099, 25)

Let us first try to build a movie recommendation system using description and taglines. we do not have a qualitative metrics to judge the machine learning perfromace hence we need to do it qualitatively.

## Movie description based recommendation system

In [26]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [27]:
tf_idf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfidf_mat = tf_idf.fit_transform(smd['description'])

In [28]:
tfidf_mat.shape

(9099, 268124)

### Cosine similarity
Cosine Similarity
I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows:

cosine(x,y)=x.y⊺||x||.||y|| 

Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster.

In [29]:
cosine_similar = linear_kernel(tfidf_mat,tfidf_mat)

In [30]:
cosine_similar[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

Now, we have a pair wise cosine similarity matrix for all the movies in our dataset. Next step is to give the top 30 movies based on the cosine similarity matrix

In [31]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index,index=smd['title'])

In [32]:
def get_recommendations(title):
    try: 
        idx = indices[title]
        sim_scores = list(enumerate(cosine_similar[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:31]
        movie_indices = [i[0] for i in sim_scores]
        return titles.iloc[movie_indices]
    except:
        print("Please check the spelling misatke")

We're all set. Let us now try and get the top recommendations for a few movies and see how good the recommendations are

In [33]:
get_recommendations('Stepmom')

443                   What Happened Was...
1155                          First Strike
5875                             Who Am I?
6882                              Semi-Pro
1309                      The House of Yes
2304                         Armour of God
5835                        State of Grace
1361                          Jackie Brown
7618                    Life As We Know It
6834                       The Bucket List
5096                     Against the Ropes
8392                           Enough Said
8150                   Killing Them Softly
2650                               The Kid
4935                      A Midnight Clear
7219                                    Up
4910                Hannah and Her Sisters
4335                           Raggedy Man
2303    Armour of God II: Operation Condor
0                                Toy Story
7363                          Grey Gardens
6210                      New Police Story
5786           Snake in the Eagle's Shadow
7737       

In [34]:
get_recommendations('Manhattan').reset_index(drop=True)

0                  What's Up, Tiger Lily?
1              Woody Allen: A Documentary
2                          Wild Man Blues
3                         Blame It on Rio
4                                Dog Park
5                              Radio Days
6                                 Sleeper
7                            The Wackness
8                              Milk Money
9           Win a Date with Tad Hamilton!
10               Smiles of a Summer Night
11    Stanley Kubrick: A Life in Pictures
12                                 Splash
13                            Match Point
14                While You Were Sleeping
15                           An Education
16                           Pajama Party
17                   Just One of the Guys
18                       Hollywood Ending
19                              Condorman
20                             Dedication
21            I Could Never Be Your Woman
22                              Toy Story
23                              De

## Metadata based recommender

In [35]:
cred = pd.read_csv('../input/the-movies-dataset/credits.csv')
key_word = pd.read_csv('../input/the-movies-dataset/keywords.csv')

In [36]:
cred.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [37]:
key_word.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [38]:
cred['id'] = cred['id'].astype('int')
key_word['id'] = key_word['id'].astype('int')
meta['id'] = meta['id'].astype('int')

In [39]:
meta = meta.merge(cred,on='id')
meta = meta.merge(key_word,on='id')

In [40]:
smd = meta[meta['id'].isin(link)]
smd.shape

(9219, 28)

In [41]:
smd.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'year', 'cast', 'crew', 'keywords'],
      dtype='object')

In [42]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x:len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [43]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [44]:
smd['director'] = smd['crew'].apply(get_director)

In [45]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])

In [46]:
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x)>=3 else x)

In [47]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])

In [48]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])

In [49]:
smd['director'] = smd['director'].astype(str).apply(lambda x: str.lower(x.replace(" ","")))

In [50]:
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [51]:
smd['director'][1]

['joejohnston', 'joejohnston', 'joejohnston']

We will do some amount of preprocessing for our keywords before putting them to any use. As a first step, we calculate the frequant counts of every keyword apperas in dataset

In [52]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [53]:
s = s.value_counts()
s

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
                       ... 
noah                      1
chevrolet                 1
charles vii.              1
immoderateness            1
decipherment              1
Name: keyword, Length: 12940, dtype: int64

In [54]:
s = s[s>1]

In [55]:
stemmer = SnowballStemmer('english')
stemmer.stem('cows')

'cow'

In [56]:
smd['keywords'][1]

['board game',
 'disappearance',
 "based on children's book",
 'new home',
 'recluse',
 'giant insect']

In [57]:
def filter_keywords(x):
    words=[]
    for i in x:
        if i in s:
            words.append(i)
    return words

In [58]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])

In [59]:
smd['soup'] = smd['director'] + smd['cast'] + smd['keywords'] + smd['genres']

In [60]:
smd['soup'] = smd['soup'].apply(lambda x: " ".join(x))

In [61]:
Count_vect = CountVectorizer(analyzer='word',min_df=0,ngram_range=(1,2),stop_words='english')
Count_vect_matrix = Count_vect.fit_transform(smd['soup'])

In [62]:
Count_vect_matrix.shape

(9219, 106737)

In [63]:
cosine_similar = cosine_similarity(Count_vect_matrix,Count_vect_matrix)

In [64]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index,index=smd['title'])

In [65]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [66]:
get_recommendations('Toy Story').head(10)

6386                Luxo Jr.
2522             Toy Story 2
7914                  Cars 2
1883            A Bug's Life
6496                    Cars
8519    Toy Story of Terror!
7629             Toy Story 3
2751       Creature Comforts
3833          Monsters, Inc.
1432        Meet the Deedles
Name: title, dtype: object

In [72]:
get_recommendations('Adventures in Babysitting').head(10)

2388                       Home Alone 2: Lost in New York
164                                           Nine Months
7538    Percy Jackson & the Olympians: The Lightning T...
2553                                     Bicentennial Man
8996                                               Pixels
1958                                              Stepmom
519                                            Home Alone
447                                        Mrs. Doubtfire
7377                              I Love You, Beth Cooper
3840             Harry Potter and the Philosopher's Stone
Name: title, dtype: object

In [77]:
def improved_recommendation(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similar[idx]))
    sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True)
    sim_scores = sim_scores[1:20]
    move_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[move_indices][['title','vote_count','vote_average','year']]
    vote_count = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_average = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_average.mean()
    m = vote_count.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(weighted_avg,axis=1)
    qualified = qualified.sort_values(by='wr',ascending=False).head(10)
    return qualified

In [79]:
improved_recommendation('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
1134,Batman Returns,1706,6,1992,5.846862
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943


In [81]:
improved_recommendation('Harry Potter and the Chamber of Secrets')

,title,vote_count,vote_average,year,wr
3840,Harry Potter and the Philosopher's Stone,7188,7,2001,6.900064
7921,Harry Potter and the Deathly Hallows: Part 2,6141,7,2011,6.884150
5452,Harry Potter and the Prisoner of Azkaban,6037,7,2004,6.882288
6354,Harry Potter and the Goblet of Fire,5758,7,2005,6.876984
7742,Harry Potter and the Deathly Hallows: Part 1,5708,7,2010,6.875983
6801,Harry Potter and the Order of the Phoenix,5633,7,2007,6.874450
7345,Harry Potter and the Half-Blood Prince,5435,7,2009,6.870214
8996,Pixels,2564,5,2015,5.035452


In [83]:
improved_recommendation('The Terminator')

,title,vote_count,vote_average,year,wr
7488,Avatar,12114,7,2009,6.939296
1376,Titanic,7770,7,1997,6.907153
522,Terminator 2: Judgment Day,4274,7,1991,6.838208
974,Aliens,3282,7,1986,6.795018
6905,I Am Legend,4977,6,2007,5.939435
4651,The Matrix Reloaded,3500,6,2003,5.916697
7296,Terminator Salvation,2496,5,2009,5.036275
8854,Terminator Genisys,3677,5,2015,5.025854


# Collaborative filtering

Our content based engine suffers from some severe limitations. It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.

Also, the engine that we built is not really personal in that it doesn't capture the personal tastes and biases of a user. Anyone querying our engine for recommendations based on a movie will receive the same recommendations for that movie, regardless of who s/he is.

Therefore, in this section, we will use a technique called Collaborative Filtering to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

I will not be implementing Collaborative Filtering from scratch. Instead, I will use the Surprise library that used extremely powerful algorithms like Singular Value Decomposition (SVD) to minimise RMSE (Root Mean Square Error) and give great recommendations.

In [87]:
reader = Reader()

In [88]:
rat = pd.read_csv('../input/the-movies-dataset/ratings_small.csv')
rat.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [89]:
rat.shape

(100004, 4)

In [91]:
rat['userId'].value_counts()

547    2391
564    1868
624    1735
15     1700
73     1610
       ... 
444      20
438      20
583      20
249      20
399      20
Name: userId, Length: 671, dtype: int64

In [98]:
data = Dataset.load_from_df(rat[['userId', 'movieId', 'rating']], reader)

In [99]:
svd = SVD()
cross_validate(svd,data,measures=['RMSE','MSE'],cv=5)

{'test_rmse': array([0.90119584, 0.89671115, 0.90081042, 0.8988403 , 0.89240251]),
 'test_mse': array([0.81215394, 0.80409088, 0.81145942, 0.80791388, 0.79638224]),
 'fit_time': (4.374215602874756,
  4.416611194610596,
  4.4214324951171875,
  4.415685176849365,
  4.5385754108428955),
 'test_time': (0.13730120658874512,
  0.13346028327941895,
  0.13382935523986816,
  0.13419437408447266,
  0.1382589340209961)}

In [101]:
from surprise.model_selection import train_test_split

In [103]:
trainset,testset = train_test_split(data,test_size=0.25)

In [106]:
algorithm = SVD()
algorithm.fit(trainset)
predictions = algorithm.test(testset)

In [108]:
algorithm.predict(5000,400,3)

Prediction(uid=5000, iid=400, r_ui=3, est=3.543598256069757, details={'was_impossible': False})

For movie with ID 302, we get an estimated prediction of 2.686. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie